In [ ]:
mari, juksemari = Contributor.objects.filter(display_name__icontains='Haanshus')

In [1]:
#finne personer som kan merges

from fuzzywuzzy import process
from django.db.models import Count
contributors = Contributor.objects.annotate(
        num_bylines=Count('byline')
    ).order_by('-num_bylines')

#one_hit_wonders = contributors.filter(num_bylines=1)
one_hit_wonders = []
with open('one_hit_wonders.txt') as fp:
    for line in fp.readlines():
        id = int(line.split()[0])
        one_hit_wonders.append(id)
        
    

In [2]:
names = sorted(contributors.values_list('display_name', 'num_bylines'

for n in names:
    print(n)
    


SyntaxError: invalid syntax (<ipython-input-2-28e44b990f82>, line 3)

In [3]:
nn = Contributor.objects.get(display_name="Nomen Nescio")
for bl in nn.byline_set.all():
    bylines = bl.story.byline_set.all()
    for byline in bylines:
        print(byline) 


@bl: Av: Nomen Nescio, KS)
@bl: Av: Nomen Nescio, LR)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, NH)
@bl: Av: Nomen Nescio, PVRH)
@bl: Av: Nomen Nescio, IKL)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen Nescio, Kjetil Strømme)
@bl: Av: Nomen Nescio, IKL)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, IKL)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, BHK)
@bl: Tekst: Kjetil Strømme)
@bl: Foto: Berit Andreassen)
@bl: Av: Nomen Nescio, @maging: – Kan jeg skrive på russisk? undrer Nils Lid Hjort, professor i matematikk. Student Lars Sydnes sier ingen ting, han er allerede godt i gang med sin ode til kvinnen)
@bl: Av: Nomen Nescio, PVRH)
@bl: Av: Nomen Nescio, TL)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, Manus og regi: Bjarne Henning Kvåle. Kamera: Khai Nguyen)
@bl: Av: Nomen Nescio, PVRH)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, PHP)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen N

@bl: Av: Nomen Nescio, SEH)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, SEH)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, SB)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, SB)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, by: Naboens tre naboer (ved en av dem))
@bl: Av: Nomen Nescio, by: Leder i Studentparlamentet i Oslo Ingrid Stranger–Thorsen)
@bl: Av: Nomen Nescio, ES)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, by: Lilly Susanne Stensland ( Foto: Balkaran Singh)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, IMJ)
@bl: Av: Nomen Nescio, IK)
@bl: Av: Nomen Nescio, IMJ)
@bl: Av: Nomen Nescio, IK)
@bl: Av: Nomen Nescio, by: Naboens tre naboer (ved en av dem))
@bl: Av: Nomen Nescio, by: Professor Emeritus Esther Bøk skriver for Universitas om vitenskap som forbigås 

@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, by: Finn H. Eriksen (En av dem. Hemmelig))
@bl: Av: Nomen Nescio, text and styling: Camilla Svendsen Skriung Foto: Åshild Bekke Eidem Modeller: Dina Thomassen)
@bl: Av: Lance Morgas)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nesci

@bl: Av: Lektorprogrammets Programutvalg, ved leiar Ane Krogsæter Aarre)
@bl: Av: Nomen Nescio, by: Henrik Asheim er leder i Unge Høyre og gjesteskribent i Universitas)
@bl: Av: Sine Astad, leder av studentforeningen ved Handelshøyskolen BI i Oslo)
@bl: Av: Nomen Nescio, SBIO)
@bl: Tekst: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, by: Bibliotek for humaniora og samfunnsvitenskap: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, UB)
@bl: Tekst: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, by: Bibliotek for humaniora og samfunnsvitenskap: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, UB)
@bl: Av: Nomen Nescio, by: Iver Aastebøl er leder i Rød Ungdom og gjesteskribent i Universitas)
@bl: Av: Nomen Nescio, by: Velferdstingets Arbeidsutvalg ved leder Mari Berdal Djupvik)
@bl: Av: Nomen Nescio, by: Aksel Braanen Sterri er leder i Studentparlamentet ved 

In [ ]:
for id in one_hit_wonders[:10]:
    person = Contributor.objects.get(pk=id)
    story = person.byline_set.first().story
    #print(story)
    print(story.id)
    print(person)
    print(json.loads(story.legacy_html_source)[0]['fields']['byline'])
    print(story.legacy_prodsys_source)

In [ ]:
import json
list(story.bylines.all())

In [ ]:
everyone = Contributor.objects.values_list('display_name', flat=True)
for contributor in contributors[:100]:
    best = process.extract(contributor.display_name, everyone, limit=2)
    print(best)
    

In [ ]:
from collections import Counter
def duplicates(queryset, attr, order_by=()):
    counter = Counter(queryset.values_list(attr, flat=True))
    duped_values = [el for el in counter if counter[el] > 1]
    sets = []
    for value in duped_values:
        items = queryset.filter(**{attr: value}).order_by(*order_by)
        sets.append(items)
    return sets

duplicates(ImageFile.objects.all(), '_size')

In [ ]:
dupe_files = [ImageFile.objects.filter(source_file=d['source_file']).order_by('-_size') for d in dupes]
for qs in dupe_files:
    print()
    for img in qs:
        img.imghash = hash_img(img)
        print('{id} {_md5} {_size:>8} {full_width:>4}x{full_height:<4} {imghash} {source_file}'.format(**img.__dict__))
    

In [ ]:
def pil_img(instance, size=8):
    with instance.source_file as fp:
        if fp.closed:
            fp.open()
        img = Image.open(fp).convert('L').resize((size,size))
    return img

def hash_img(instance):
    return str(imagehash.dhash(pil_img(instance)))


In [ ]:
from PIL import ImageChops
img1 = pil_img(imgfile, size)
img2 = cv_img(imgfile, size)
hashfuncs = imagehash.phash, imagehash.dhash, imagehash.whash
for func in hashfuncs:
    hash1 = func(img1)
    hash2 = func(img2)
    print(func.__name__, hash1, hash2, hash1-hash2)

ImageChops.difference(img1, img2).resize((200,200))

In [ ]:
%timeit imagehash.dhash(img1)
%timeit imagehash.whash(img1)
%timeit imagehash.phash(img1)

In [ ]:
im0 = cv_img(ImageFile.objects.first())
for fn in dupe_files:
    im1, im2 = ImageFile.objects.filter(source_file=fn)[:2]
    diff = mse(cv_img(im1), cv_img(im2))
    print(diff)
    

In [ ]:
!pip install --user imagehash

In [ ]:
from PIL import Image

import imagehash



In [ ]:
from sorl import thumbnail
def make_thumb(imf, width=8, height=8):
    

make_thumb(ImageFile.objects.first())

In [ ]:
_7.x

In [ ]:
from django.db import transaction
from django.apps import apps
from django.contrib.contenttypes.fields import GenericForeignKey
from django.db.models.fields.related import ManyToManyField

@transaction.atomic()
def merge_instances(primary_object, *alias_objects):
    """Merge several model instances into one, the `primary_object`.
    Use this function to merge model objects and migrate all of the related
    fields from the alias objects the primary object.
    Usage:
        from django.contrib.auth.models import User
        primary_user = User.objects.get(email='good@example.com')
        duplicate_user = User.objects.get(email='good+duplicate@example.com')
        merge(primary_user, duplicate_user)
    Based on: https://djangosnippets.org/snippets/382/
    """
    generic_fields = get_generic_fields()

    # get related fields
    many_to_many_fields, related_fields = discrimine(
        lambda field: isinstance(field, ManyToManyField),
        primary_object._meta._get_fields(forward=False, include_hidden=True)
    )

    # Loop through all alias objects and migrate their references to the
    # primary object
    for alias_object in alias_objects:
        # Migrate all foreign key references from alias object to primary
        # object.
        for related_object in related_fields:
            # The variable name on the alias_object model.
            alias_varname = related_object.get_accessor_name()
            # The variable name on the related model.
            obj_varname = related_object.field.name
            related_objects = getattr(alias_object, alias_varname)
            for obj in related_objects.all():
                setattr(obj, obj_varname, primary_object)
                obj.save()

        # Migrate all many to many references from alias object to primary
        # object.
        for related_many_object in many_to_many_fields:
            alias_varname = related_many_object.get_accessor_name()
            obj_varname = related_many_object.field.name
            related_many_objects = getattr(alias_object, alias_varname)
            for obj in related_many_objects.all():
                getattr(obj, obj_varname).remove(alias_object)
                getattr(obj, obj_varname).add(primary_object)

        # Migrate all generic foreign key references from alias object to
        # primary object.
        for field in generic_fields:
            filter_kwargs = {}
            filter_kwargs[field.fk_field] = alias_object._get_pk_val()
            filter_kwargs[field.ct_field] = field.get_content_type(alias_object)
            related_objects = field.model.objects.filter(**filter_kwargs)
            for generic_related_object in related_objects:
                setattr(generic_related_object, field.name, primary_object)
                generic_related_object.save()

        if alias_object.id:
            alias_object.delete()

    return primary_object


def get_generic_fields():
    """Return a list of all GenericForeignKeys in all models."""
    generic_fields = []
    for model in apps.get_models():
        for field_name, field in model.__dict__.items():
            if isinstance(field, GenericForeignKey):
                generic_fields.append(field)
    return generic_fields


def discrimine(pred, sequence):
    """Split a collection in two collections using a predicate.
    >>> discrimine(lambda x: x < 5, [3, 4, 5, 6, 7, 8])
    ... ([3, 4], [5, 6, 7, 8])
    """
    positive, negative = [], []
    for item in sequence:
        if pred(item):
            positive.append(item)
        else:
            negative.append(item)
    return positive, negative
